In [4]:
import sys
sys.path.append("../src/") 
import os



import model_manipulation as mm
import cobra
import cplex 
import libsbml
import pandas as pd
import copy
from pathlib import Path
import matplotlib.pyplot as plt
import csv
import numpy as np
import seaborn as sns
from cobra import Reaction
import model_initialize as model_init
from model_initialize import compute_ngam_atp, define_model_medium, add_tissue_constraints, add_enzyme_constraints, \
                        add_ngam_cons, print_summary, turn_off_cofac_cycles

Checklist:
- add wild type script to this script //
- Change names to reflect WT and trans //
- Incorporate graphs for both WT and trans//

- Check WT FVA @ 1000 PPFD to check CO2 intake and base CO2 script from there.



In [2]:
#Read 2-cell model
model = cobra.io.read_sbml_model("../model/ios2164_2cell.xml")
model.solver = 'gurobi'

inf = 1e6

Set parameter Username
Academic license - for non-commercial use only - expires 2023-08-03


One issue that I've found with my model is that allowing intake of H+ increases biomass yield per light value drastically. Furthermore, I do think that not allowing protons to be taken into the model is erroneoous and should be rectified.

One approach that I could do with this is to allow excess protons to exit the systems via sink reactions in each compartment. THis approach was adopted by Blaetke and Brautigam (2019) 


In [5]:

#This 
ppfd_scan_h1 = pd.DataFrame(index=list(i.id for i in model.reactions))

#Iterate from range 50 to 1500 w/ increments of 50 (max ppfd)

#Note the following constraints:
#CO2 intake limited to 22.2 

for ppfd in range(50,1550,50):
    
    #Generate instance of model every iteration
    with model as m1:
        
        #Turn on proton sinks
        for rxns in m1.reactions:
            if "DM_h" in rxns.id:
                m1.reactions.get_by_id(rxns.id).upper_bounds = inf
#                 print(rxns, rxns.upper_bounds)
        
 
        m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_tissue_constraints(m1)
        add_enzyme_constraints(m1) 
        
        #Add NGAM to model
        add_ngam_cons(m1, ppfd)
    
        #Change objective function to mature leaf
        m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        while True:
            solution = model.optimize()
            if solution.status == 'optimal':
                sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                #Append to Dataframe
                ppfd_scan_h1[ppfd] = sample_fluxes['fluxes']
                break
            

filepath = '../flux_results/Proton_benchmarking/'
filename = 'PPFD-benchmark-h-input-with-sink'
mm.save_fba_matrix(filename, ppfd_scan_h1, filepath)



PPFD: 50  RBPC_M cell flux: 7.560798572971715
PPFD: 100  RBPC_M cell flux: 12.113350189834984
PPFD: 150  RBPC_M cell flux: 16.61660924709708
PPFD: 200  RBPC_M cell flux: 20.96470255720837
PPFD: 250  RBPC_M cell flux: 25.311383298265326
PPFD: 300  RBPC_M cell flux: 29.658064039334448
PPFD: 350  RBPC_M cell flux: 31.738199559135435
PPFD: 400  RBPC_M cell flux: 31.813130857845955
PPFD: 450  RBPC_M cell flux: 31.860572047879334
PPFD: 500  RBPC_M cell flux: 31.76001399371039
PPFD: 550  RBPC_M cell flux: 31.99033670005153
PPFD: 600  RBPC_M cell flux: 31.94962250803559
PPFD: 650  RBPC_M cell flux: 31.952988603170844
PPFD: 700  RBPC_M cell flux: 31.888948595590183
PPFD: 750  RBPC_M cell flux: 31.826211550807937
PPFD: 800  RBPC_M cell flux: 31.822973223751028
PPFD: 850  RBPC_M cell flux: 31.84433368915516
PPFD: 900  RBPC_M cell flux: 31.841903911517058
PPFD: 950  RBPC_M cell flux: 31.8405445325654
PPFD: 1000  RBPC_M cell flux: 31.839359623020425
PPFD: 1050  RBPC_M cell flux: 31.838174713485177


In [6]:
ppfd_scan_h2 = pd.DataFrame(index=list(i.id for i in model.reactions))


for ppfd in range(50,1550,50):
    
    #Generate instance of model every iteration
    with model as m1:
        
        #Turn on proton sinks
        for rxns in m1.reactions:
            if "DM_h" in rxns.id:
                m1.reactions.get_by_id(rxns.id).upper_bounds = inf
#                 print(rxns, rxns.upper_bounds)
        
 
        m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=0, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_tissue_constraints(m1)
        add_enzyme_constraints(m1) 
        
        #Add NGAM to model
        add_ngam_cons(m1, ppfd)
    
        #Change objective function to mature leaf
        m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        while True:
            solution = model.optimize()
            if solution.status == 'optimal':
                sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                #Append to Dataframe
                ppfd_scan_h2[ppfd] = sample_fluxes['fluxes']
                break


filepath = '../flux_results/Proton_benchmarking/'
filename = 'PPFD-benchmark-no-proton-input-with-sink'
mm.save_fba_matrix(filename, ppfd_scan_h2, filepath)

PPFD: 50  RBPC_M cell flux: 1.2124690857353373
PPFD: 100  RBPC_M cell flux: 3.124166412964314
PPFD: 150  RBPC_M cell flux: 5.052795681867245
PPFD: 200  RBPC_M cell flux: 6.879403519036243
PPFD: 250  RBPC_M cell flux: 8.83550654020474
PPFD: 300  RBPC_M cell flux: 10.748393068287651
PPFD: 350  RBPC_M cell flux: 12.664636479201219
PPFD: 400  RBPC_M cell flux: 14.580015646573548
PPFD: 450  RBPC_M cell flux: 16.484758846926635
PPFD: 500  RBPC_M cell flux: 18.396914513450323
PPFD: 550  RBPC_M cell flux: 20.329020450609686
PPFD: 600  RBPC_M cell flux: 22.24877195302377
PPFD: 650  RBPC_M cell flux: 24.170996545507546
PPFD: 700  RBPC_M cell flux: 26.087328275803838
PPFD: 750  RBPC_M cell flux: 28.0036600064469
PPFD: 800  RBPC_M cell flux: 29.91999173640688
PPFD: 850  RBPC_M cell flux: 31.56793326627277
PPFD: 900  RBPC_M cell flux: 31.79767217556516
PPFD: 950  RBPC_M cell flux: 31.795926114182915
PPFD: 1000  RBPC_M cell flux: 31.7941800527992
PPFD: 1050  RBPC_M cell flux: 31.792433991332395
PPFD

In [7]:
ppfd_scan_h3 = pd.DataFrame(index=list(i.id for i in model.reactions))


for ppfd in range(50,1550,50):
    
    #Generate instance of model every iteration
    with model as m1:
        
        #Turn on proton sinks
#         for rxns in m1.reactions:
#             if "DM_h" in rxns.id:
#                 rxns.upper_bounds = inf
#                 print(rxns, rxns.upper_bounds)
        
 
        m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=inf, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_tissue_constraints(m1)
        add_enzyme_constraints(m1) 
        
        #Add NGAM to model
        add_ngam_cons(m1, ppfd)
    
        #Change objective function to mature leaf
        m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        while True:
            solution = model.optimize()
            if solution.status == 'optimal':
                sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                #Append to Dataframe
                ppfd_scan_h3[ppfd] = sample_fluxes['fluxes']
                break


filepath = '../flux_results/Proton_benchmarking/'
filename = 'PPFD-benchmark-with-proton-input-no-sink'
mm.save_fba_matrix(filename, ppfd_scan_h3, filepath)

PPFD: 50  RBPC_M cell flux: 7.560798572971398
PPFD: 100  RBPC_M cell flux: 12.11335018985084
PPFD: 150  RBPC_M cell flux: 16.616609247096633
PPFD: 200  RBPC_M cell flux: 20.964702557204266
PPFD: 250  RBPC_M cell flux: 25.311383298270314
PPFD: 300  RBPC_M cell flux: 29.65806403933396
PPFD: 350  RBPC_M cell flux: 31.712634463268827
PPFD: 400  RBPC_M cell flux: 31.928797076114936
PPFD: 450  RBPC_M cell flux: 31.8464265603779
PPFD: 500  RBPC_M cell flux: 31.82126852941287
PPFD: 550  RBPC_M cell flux: 32.010123382105256
PPFD: 600  RBPC_M cell flux: 32.01330243187956
PPFD: 650  RBPC_M cell flux: 31.95298860317089
PPFD: 700  RBPC_M cell flux: 31.854482898463836
PPFD: 750  RBPC_M cell flux: 31.826211550807642
PPFD: 800  RBPC_M cell flux: 31.82297322375059
PPFD: 850  RBPC_M cell flux: 31.84433368915453
PPFD: 900  RBPC_M cell flux: 31.84190391105468
PPFD: 950  RBPC_M cell flux: 31.84054453255406
PPFD: 1000  RBPC_M cell flux: 31.839359623019984
PPFD: 1050  RBPC_M cell flux: 31.838174713486644
PPF

In [8]:
ppfd_scan_h4 = pd.DataFrame(index=list(i.id for i in model.reactions))


for ppfd in range(50,1550,50):
    
    #Generate instance of model every iteration
    with model as m1:
        
        #Turn on proton sinks
#         for rxns in m1.reactions:
#             if "DM_h" in rxns.id:
#                 rxns.upper_bounds = inf
#                 print(rxns, rxns.upper_bounds)
        
 
        m1.medium = define_model_medium(m1, co2=29, o2=2.26, ppfd=ppfd, h=0, nh4=inf, no3=inf)
        turn_off_cofac_cycles(m1) #Turn off other cofactor recycling 
        add_tissue_constraints(m1)
        add_enzyme_constraints(m1) 
        
        #Add NGAM to model
        add_ngam_cons(m1, ppfd)
    
        #Change objective function to mature leaf
        m1.reactions.get_by_id('Straw_Biomass_M').objective_coefficient = 0
        m1.reactions.get_by_id('Straw_Biomass_BS').objective_coefficient = 0
        m1.reactions.get_by_id('DM_Phloem_BS').objective_coefficient = 1

        #Optimize then check optimality
        while True:
            solution = model.optimize()
            if solution.status == 'optimal':
                sample_fluxes = cobra.flux_analysis.pfba(model).fluxes
                #Remove thermodynamically infeasible loops via CycleFreeFlux algorithm
                sample_fluxes = cobra.flux_analysis.loopless_solution(model, sample_fluxes).to_frame()
                print("PPFD:", ppfd, " RBPC_M cell flux:", sample_fluxes['fluxes']['RBPCs_M'])
                #Append to Dataframe
                ppfd_scan_h4[ppfd] = sample_fluxes['fluxes']
                break


filepath = '../flux_results/Proton_benchmarking/'
filename = 'PPFD-benchmark-no-proton-input-no-sink'
mm.save_fba_matrix(filename, ppfd_scan_h4, filepath)

PPFD: 50  RBPC_M cell flux: 1.2178128535142692
PPFD: 100  RBPC_M cell flux: 3.1349412766960514
PPFD: 150  RBPC_M cell flux: 5.052795681866689
PPFD: 200  RBPC_M cell flux: 6.879403519034048
PPFD: 250  RBPC_M cell flux: 8.835506540208197
PPFD: 300  RBPC_M cell flux: 10.748393068287331
PPFD: 350  RBPC_M cell flux: 12.664636479201784
PPFD: 400  RBPC_M cell flux: 14.580015646573013
PPFD: 450  RBPC_M cell flux: 16.484758846943844
PPFD: 500  RBPC_M cell flux: 18.396914513450152
PPFD: 550  RBPC_M cell flux: 20.329020450609764
PPFD: 600  RBPC_M cell flux: 22.24877195304918
PPFD: 650  RBPC_M cell flux: 24.17099654551627
PPFD: 700  RBPC_M cell flux: 26.08732827580882
PPFD: 750  RBPC_M cell flux: 28.003660006058684
PPFD: 800  RBPC_M cell flux: 29.919991736415287
PPFD: 850  RBPC_M cell flux: 31.56793326629762
PPFD: 900  RBPC_M cell flux: 31.797672175566834
PPFD: 950  RBPC_M cell flux: 31.795926114182507
PPFD: 1000  RBPC_M cell flux: 31.794180052802282
PPFD: 1050  RBPC_M cell flux: 31.79243399141501

Notes:
Adding H+ to the model allows CO2 assimilation to follow a curve similar to B&B 2019 w/ a saturation point around 400-500 ppfd.

Scenarios without added H+ have a more gradual curve with a saturation point around 700-800 ppfd